In [17]:
# AV submission 2
import math as m
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder  
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor#export graph_viz
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder  
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [18]:
df=pd.read_csv("train.csv")
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [19]:
df['Product_Category_2'].fillna(df['Product_Category_2'].mode()[0],inplace=True)
df['Product_Category_3'].fillna(df['Product_Category_2'].mode()[0],inplace=True)
label=LabelEncoder()
cat=['Gender','Age','Product_ID','City_Category','Stay_In_Current_City_Years']
for x in cat:
    df[x]=label.fit_transform(df[x])

In [20]:
df.dtypes

User_ID                         int64
Product_ID                      int32
Gender                          int32
Age                             int32
Occupation                      int64
City_Category                   int32
Stay_In_Current_City_Years      int32
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [21]:
predictor_var=list(df.columns)
rem=['Gender','Occupation','Stay_In_Current_City_Years','Marital_Status','Purchase']
for x in rem:
    predictor_var.remove(x)
outcome='Purchase'
predictor_var

['User_ID',
 'Product_ID',
 'Age',
 'City_Category',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3']

In [36]:
def Regression_model(model,data,predictors,outcome):
    #model.fit(data[predictors],data[outcome])
    kth=KFold(n_splits=5)
    kth.get_n_splits(data)
    error=[]
    for train,test in kth.split(data):
        train_predictors=(data[predictors].iloc[train,:])
        train_target=data[outcome].iloc[train]
        model.fit(train_predictors,train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
    prediction=model.predict(data[predictors])
    accuracy=mean_squared_error(prediction,data[outcome])
    print ("RMSE : %s" % "{0:.3%}".format(accuracy))
    model.fit(data[predictors],data[outcome])    

In [ ]:
#model1=DecisionTreeRegressor(random_state = 0) #4451
model=RandomForestRegressor(n_estimators=500, min_samples_split=500, max_depth=7, max_features=len(predictor_var),random_state=100)
#3260
#model=GradientBoostingRegressor(n_estimators=25,max_depth=4,min_samples_split=2,
                               # learning_rate=0.01,loss='ls')
Regression_model(model,df,predictor_var,outcome)
#feat_importance = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
#print (feat_importance)

Cross-Validation Score : 66.486%
RMSE : 842789088.632%


In [65]:
test=pd.read_csv("test.csv")
orgtest=pd.read_csv("test.csv")
test.head(1)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN


In [66]:
test['Product_Category_2'].fillna(99,inplace=True)
test['Product_Category_3'].fillna(99,inplace=True)
label=LabelEncoder()
cat=['Gender','Age','City_Category','Product_ID','Stay_In_Current_City_Years']
for x in cat:
    test[x]=label.fit_transform(test[x])

In [67]:
pred=model.predict(test[predictor_var])
finalDF=pd.DataFrame()
orgtest=orgtest.applymap(str)
finalDF['User_ID']=orgtest['User_ID']
finalDF['Product_ID']=orgtest['Product_ID']
finalDF['Purchase']=pred
finalDF.to_csv('Sol1.csv',index=False)
orgtest.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,nan
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,nan
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,nan
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,nan
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
